# NVIDIA Driver Installation Required

**⚠️ IMPORTANT: You need to install NVIDIA drivers first!**

Your system shows an NVIDIA L4 GPU but no drivers are installed. Run these commands in a terminal:

```bash
# Install the recommended NVIDIA driver
sudo apt update
sudo apt install nvidia-driver-580-open

# Reboot system (required!)
sudo reboot
```

After reboot, verify with:
```bash
nvidia-smi
```

Then you can proceed with the matchmaker setup.

In [ ]:
from matchmaker import matchmaker

In [ ]:
engine = matchmaker.MatchingEngine()
engine.load_interactions("data/swipes.csv", 
  decider_col='decidermemberid', 
  other_col='othermemberid', 
  like_col='like', 
  timestamp_col='timestamp')

In [ ]:
engine.interactions_df.head()